## Datasets

**Dataset 1: BoardGameGeek Reviews**

https://www.kaggle.com/datasets/jvanelteren/boardgamegeek-reviews

This dataset contains detailed metadata for a large collection of board games, scraped from BoardGameGeek (BGG)

Files used:

1.  **games_detailed_info2025.csv**: Contains board game metadata, scraped in 2025.
2.  **games_detailed_info.csv** (renamed to **games_detailed_info2022.csv**): Contains board game metadata scraped from BGG in 2022. The file was renamed to avoid confusion during analysis.



**Dataset 2: Board Game Ratings by Country**

https://www.kaggle.com/datasets/thedevastator/board-game-ratings-by-country

This dataset provides user ratings by country and is used to analyze global preferences from the year 2019.

Files used:

1.  **games_detailed_info.xlsx**: Converted to CSV and renamed to **games_detailed_info2019.csv**; contains board game metadata from 2019.
2.  **bggreviews_small.csv** (renamed to **ratings.csv**): Contains individual user ratings for various games.
3.  **users.csv**: Includes information about users, including their country of origin, allowing for regional analysis.

## Data Processing and Cleaning

In [ ]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import chain

In [76]:
games19 = pd.read_csv('../../data/games_detailed_info2019.csv', index_col='id') #games19['id'].is_unique evaluates to True
games22 = pd.read_csv('../../data/games_detailed_info2022.csv', index_col='id')
games25 = pd.read_csv('../../data/games_detailed_info2025.csv', index_col='id')

/var/folders/v8/zf75ytl94239zprww5z1qwfr0000gp/T/ipykernel_42691/1360892205.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  games19 = pd.read_csv('../../data/games_detailed_info2019.csv', index_col='id') #games19['id'].is_unique evaluates to True
/var/folders/v8/zf75ytl94239zprww5z1qwfr0000gp/T/ipykernel_42691/1360892205.py:2: DtypeWarning: Columns (29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  games22 = pd.read_csv('../../data/games_detailed_info2022.csv', index_col='id')


In [77]:
def align_dataframes(df1, df2, df3):
    """
    Aligns the three dataframes on common columns.
    Returns cleaned versions: df1_cleaned, df2_cleaned, df3_cleaned
    """

    def clean(df):
        df = df.copy()
        if 'Unnamed: 0' in df.columns:
            df.drop(columns=['Unnamed: 0'], inplace=True)
        if 'primary' in df.columns:
            df.rename(columns={'primary': 'name'}, inplace=True)
        return df

    #Clean each dataframe
    df1 = clean(df1)
    df2 = clean(df2)
    df3 = clean(df3)

    #Get common columns
    common_cols = set(df1.columns) & set(df2.columns) & set(df3.columns)

    #Keep only common columns
    df1 = df1[list(common_cols)]
    df2 = df2[list(common_cols)]
    df3 = df3[list(common_cols)]

    return df1, df2, df3

In [78]:
games19, games22, games25 = align_dataframes(games19, games22, games25)
games19.head(3)

,stddev,Party Game Rank,minage,RPG Item Rank,wishing,numcomments,usersrated,name,boardgamecompilation,War Game Rank,...,averageweight,alternate,Children's Game Rank,Thematic Rank,boardgamepublisher,playingtime,Board Game Rank,Abstract Game Rank,suggested_playerage,trading
id,,,,,,,,,,,,,,,,,,,,,
13,1.47121,NaN,10,NaN,4729,16330,85332,Catan,"[""CATAN 3D Collector's Edition"", 'Catan: Big B...",NaN,...,2.3399,"['CATAN', 'Catan (ÐšÐ¾Ð»Ð¾Ð½Ð¸Ð·Ð°Ñ‚Ð¾Ñ€Ñ‹)', ...",NaN,NaN,"['KOSMOS', '999 Games', 'Albi', 'Astrel Games'...",120,327,NaN,"[OrderedDict([('@value', '2'), ('@numvotes', '...",1685
822,1.30292,NaN,8,NaN,5403,16180,85052,Carcassonne,"['Carcassonne Big Box', 'Carcassonne Big Box 2...",NaN,...,1.9247,"['Carcassonne Jubilee Edition', 'Carcassonne: ...",NaN,NaN,"['Hans im GlÃ¼ck', '999 Games', 'Albi', 'Bard ...",45,151,NaN,"[OrderedDict([('@value', '2'), ('@numvotes', '...",1477
30549,1.32212,NaN,8,NaN,7655,14127,83486,Pandemic,NaN,NaN,...,2.4220,"['EPIZOotic', 'Pandemia', 'Pandemia 10 Anivers...",NaN,NaN,"['Z-Man Games, Inc.', '(Unknown)', 'Albi', 'As...",45,74,NaN,"[OrderedDict([('@value', '2'), ('@numvotes', '...",1977


The cleaned dataframes contain 48 columns, but not all of them are relevant to our analysis. Below are the main columns we will keep and use:

- **id**: Unique BoardGameGeek identifier for each game
- **name**: The name of the game
- **yearpublished**: The year the game was originally published
- **Board Game Rank** (renamed to **rank**): The official rank of the game on BoardGameGeek
- **bayesaverage**: The Bayesian-adjusted average rating. We will use this instead of the simple average, as it provides a more balanced and fair representation of a game's quality
- **usersrated**: Number of users who rated the game
- **averageweight**: A measure of the game's complexity on a 5-point scale from 1 (Light) to 5 (Heavy), indicating how difficult it is to understand and play
- **minplayers** and **maxplayers**: The minimum and maximum number of players supported
- **minplaytime** and **maxplaytime**: Minimum and maximum estimated playtime
- **minage**: Minimum recommended age for players
- **boardgamecategory**: The game's type. This column will be cleaned and grouped into broader categories for analysis
- **numcomments**: Number of user comments. This can be used to measure engagement or highlight top games on the website
- **owned**. **wishing**, **trading**: These columns reflect different stages of user engagement with a game. **owned** indicates how many users have the game in their collection, **wishing** shows how many users want to own it, and **trading** reflects how many users are offering it for trade
- **description**: Used as a preview on the website for top games
- **image**: Game cover image, used for visual display on the website
- **boardgameexpansion**: Indicates the expansions of a game. This can be used to analyze if top-ranked games tend to have expansions


In [79]:
columns_to_keep = [
    'id', 'name', 'yearpublished', 'Board Game Rank', 'bayesaverage',
    'usersrated', 'averageweight', 'minplayers', 'maxplayers',
    'minplaytime', 'maxplaytime', 'minage', 'boardgamecategory',
    'numcomments', 'owned', 'wishing', 'trading',
    'description', 'image', 'boardgameexpansion'
]

#Keep only those columns
games19 = games19[[col for col in columns_to_keep if col in games19.columns]].copy()
games22 = games22[[col for col in columns_to_keep if col in games22.columns]].copy()
games25 = games25[[col for col in columns_to_keep if col in games25.columns]].copy()

#Rename 'Board Game Rank' to 'rank'
games19.rename(columns={'Board Game Rank': 'rank'}, inplace=True)
games22.rename(columns={'Board Game Rank': 'rank'}, inplace=True)
games25.rename(columns={'Board Game Rank': 'rank'}, inplace=True)


In [80]:
def dataframe_summary(df, name):
    """
    Returns a detailed summary of nulls, empty strings, and data types. (similar to .info() but with more information on empty string columns and the %)
    """

    summary = pd.DataFrame(columns=[
        'Column', 'Data Type', 'Total', 'Non-Null',
        'Null %', 'Empty String %'
    ])

    for col in df.columns:
        total = len(df)
        non_null = df[col].notna().sum()
        null_pct = round(((total - non_null) / total) * 100, 2)
        empty_pct = round((df[col].astype(str).str.strip() == '').sum() / total * 100, 2)
        dtype = df[col].dtype

        summary = pd.concat([summary, pd.DataFrame({
            'Column': [col],
            'Data Type': [dtype],
            'Total': [total],
            'Non-Null': [non_null],
            'Null %': [null_pct],
            'Empty String %': [empty_pct]
        })], ignore_index=True)

    print(f"\nDetailed Summary for {name}:\n")
    return summary

In [81]:
dataframe_summary(games19,"games19")


Detailed Summary for games19:



/var/folders/v8/zf75ytl94239zprww5z1qwfr0000gp/T/ipykernel_42691/4034920259.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary = pd.concat([summary, pd.DataFrame({


,Column,Data Type,Total,Non-Null,Null %,Empty String %
0,name,object,17063,17063,0.00,0.0
1,yearpublished,int64,17063,17063,0.00,0.0
2,rank,object,17063,17063,0.00,0.0
3,bayesaverage,float64,17063,17063,0.00,0.0
4,usersrated,int64,17063,17063,0.00,0.0
5,averageweight,float64,17063,17063,0.00,0.0
6,minplayers,int64,17063,17063,0.00,0.0
7,maxplayers,int64,17063,17063,0.00,0.0
8,minplaytime,int64,17063,17063,0.00,0.0
9,maxplaytime,int64,17063,17063,0.00,0.0


In [82]:
dataframe_summary(games22,"games22")


Detailed Summary for games22:



/var/folders/v8/zf75ytl94239zprww5z1qwfr0000gp/T/ipykernel_42691/4034920259.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary = pd.concat([summary, pd.DataFrame({


,Column,Data Type,Total,Non-Null,Null %,Empty String %
0,name,object,21631,21631,0.00,0.0
1,yearpublished,int64,21631,21631,0.00,0.0
2,rank,object,21631,21631,0.00,0.0
3,bayesaverage,float64,21631,21631,0.00,0.0
4,usersrated,int64,21631,21631,0.00,0.0
5,averageweight,float64,21631,21631,0.00,0.0
6,minplayers,int64,21631,21631,0.00,0.0
7,maxplayers,int64,21631,21631,0.00,0.0
8,minplaytime,int64,21631,21631,0.00,0.0
9,maxplaytime,int64,21631,21631,0.00,0.0


In [83]:
dataframe_summary(games25,"games25")


Detailed Summary for games25:



/var/folders/v8/zf75ytl94239zprww5z1qwfr0000gp/T/ipykernel_42691/4034920259.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary = pd.concat([summary, pd.DataFrame({


,Column,Data Type,Total,Non-Null,Null %,Empty String %
0,name,object,27780,27780,0.00,0.0
1,yearpublished,int64,27780,27780,0.00,0.0
2,rank,int64,27780,27780,0.00,0.0
3,bayesaverage,float64,27780,27780,0.00,0.0
4,usersrated,int64,27780,27780,0.00,0.0
5,averageweight,float64,27780,27780,0.00,0.0
6,minplayers,int64,27780,27780,0.00,0.0
7,maxplayers,int64,27780,27780,0.00,0.0
8,minplaytime,int64,27780,27780,0.00,0.0
9,maxplaytime,int64,27780,27780,0.00,0.0


We have values for most of the selected columns. The only attribute with missing data is boardgameexpansion, but the number of available samples is sufficient to include it in our analysis.

In [84]:
def clean_dataframe(df):
    """
    Cleans and parses a dataframe:
    - Parses list columns
    - Converts 'rank' to integer
    - Ensures 'name' and 'description' are strings
    - Drops rows with empty or missing boardgamecategory, rank, yearpublished, or weight
    """
    df = df.copy()

    #Convert 'name' and 'description' to strings
    df['name'] = df['name'].astype(str)
    df['description'] = df['description'].astype(str)

    #Drop rows where rank is "Not Ranked"
    df = df[df['rank'] != 'Not Ranked']

    #Convert 'rank' to integer
    df['rank'] = pd.to_numeric(df['rank'], errors='coerce')

    #Drop rows with missing rank (after conversion)
    df = df[df['rank'].notna()]
    df['rank'] = df['rank'].astype('int64')

    #Drop rows with yearpublished=0 (missing years)
    df = df[df['yearpublished'] != 0]

    #Drop rows with averageweight less than 1 (missing weight)
    df = df[df['averageweight'] != 0]

    #Convert 'boardgamecategory' to list
    df['boardgamecategory'] = df['boardgamecategory'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else []
    )

    #Convert 'boardgameexpansion' to list
    df['boardgameexpansion'] = df['boardgameexpansion'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else []
    )

    #Drop rows with empty or null boardgamecategory
    df = df[df['boardgamecategory'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

    return df

In [85]:
games19 = clean_dataframe(games19)
games22 = clean_dataframe(games22)
games25 = clean_dataframe(games25)

In [86]:
#Re-ordering columns for better readability

ordered_columns = [
    'name', 'yearpublished', 'rank',
    'bayesaverage', 'usersrated', 'numcomments',
    'owned', 'wishing', 'trading',
    'averageweight',
    'minplayers', 'maxplayers',
    'minplaytime', 'maxplaytime',
    'minage',
    'boardgamecategory', 'boardgameexpansion',
    'description', 'image'
]

games19 = games19[ordered_columns]
games22 = games22[ordered_columns]
games25 = games25[ordered_columns]

In [87]:
games19.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16449 entries, 13 to 8256
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                16449 non-null  object 
 1   yearpublished       16449 non-null  int64  
 2   rank                16449 non-null  int64  
 3   bayesaverage        16449 non-null  float64
 4   usersrated          16449 non-null  int64  
 5   numcomments         16449 non-null  int64  
 6   owned               16449 non-null  int64  
 7   wishing             16449 non-null  int64  
 8   trading             16449 non-null  int64  
 9   averageweight       16449 non-null  float64
 10  minplayers          16449 non-null  int64  
 11  maxplayers          16449 non-null  int64  
 12  minplaytime         16449 non-null  int64  
 13  maxplaytime         16449 non-null  int64  
 14  minage              16449 non-null  int64  
 15  boardgamecategory   16449 non-null  object 
 16  boardgame

In [88]:
games19.head(5)

,name,yearpublished,rank,bayesaverage,usersrated,numcomments,owned,wishing,trading,averageweight,minplayers,maxplayers,minplaytime,maxplaytime,minage,boardgamecategory,boardgameexpansion,description,image
id,,,,,,,,,,,,,,,,,,,
13,Catan,1995,327,7.03556,85332,16330,125254,4729,1685,2.3399,3,4,60,120,10,[Negotiation],"[20 Jahre Darmstadt Spielt, Brettspiel Advents...","In Catan (formerly The Settlers of Catan), pla...",https://cf.geekdo-images.com/original/img/A-0y...
822,Carcassonne,2000,151,7.32084,85052,16180,121717,5403,1477,1.9247,2,5,30,45,8,"[City Building, Medieval, Territory Building]","[20 Jahre Darmstadt Spielt, Apothecaries And T...",Carcassonne is a tile-placement game in which ...,https://cf.geekdo-images.com/original/img/o4p6...
30549,Pandemic,2008,74,7.54432,83486,14127,123067,7655,1977,2.4220,2,4,45,45,8,[Medical],[Pandemic: Gen Con 2016 Promos â€“ Z-Force Tea...,"In Pandemic, several virulent diseases have br...",https://cf.geekdo-images.com/original/img/j-pf...
68448,7 Wonders,2010,45,7.68858,69132,11816,91037,9464,1162,2.3397,2,7,30,30,10,"[Ancient, Card Game, City Building, Civilization]","[7 Wonders: Armada, 7 Wonders: Babel, 7 Wonder...",You are the leader of one of the 7 great citie...,https://cf.geekdo-images.com/original/img/3DP_...
36218,Dominion,2008,76,7.54296,68284,12178,88729,6703,1806,2.3623,2,4,30,30,13,"[Card Game, Medieval]",[Animals expansion (mini fan expansion for Dom...,(from the back of the box:)&#10;&#10;&quot;You...,https://cf.geekdo-images.com/original/img/oN8C...


Now that we are done processing the board game dataframes, we will need to process the users and ratings dataframes.

In [89]:
users = pd.read_csv('../../data/users.csv', encoding='latin-1')
ratings = pd.read_csv('../../data/ratings.csv')

In [90]:
users

,users,url,country
0,sidehacker,https://boardgamegeek.com/user/sidehacker,United States
1,Varthlokkur,https://boardgamegeek.com/user/Varthlokkur,United States
2,dougthonus,https://boardgamegeek.com/user/dougthonus,United States
3,cypar7,https://boardgamegeek.com/user/cypar7,United States
4,ssmooth,https://boardgamegeek.com/user/ssmooth,United States
...,...,...,...
289909,stevedstreet,https://boardgamegeek.com/user/stevedstreet,NaN
289910,cva6633,https://boardgamegeek.com/user/cva6633,Canada
289911,carlospadino,https://boardgamegeek.com/user/carlospadino,United States
289912,Paultiful,https://boardgamegeek.com/user/Paultiful,Germany


In [91]:
ratings

,user,rating,ID
0,sidehacker,10.0,13
1,Varthlokkur,10.0,13
2,dougthonus,10.0,13
3,cypar7,10.0,13
4,ssmooth,10.0,13
...,...,...,...
13170068,pswissler,3.0,8256
13170069,Sprayoncrayon,3.0,8256
13170070,PhoenixSong,2.5,8256
13170071,tsantos,2.0,8256


In [92]:
ratings = ratings.rename(columns={"user": "users"})

#Add country of the user to their rating
ratings_by_country = pd.merge(ratings, users, on="users", how="inner")
ratings_by_country

,users,rating,ID,url,country
0,sidehacker,10.0,13,https://boardgamegeek.com/user/sidehacker,United States
1,Varthlokkur,10.0,13,https://boardgamegeek.com/user/Varthlokkur,United States
2,dougthonus,10.0,13,https://boardgamegeek.com/user/dougthonus,United States
3,cypar7,10.0,13,https://boardgamegeek.com/user/cypar7,United States
4,ssmooth,10.0,13,https://boardgamegeek.com/user/ssmooth,United States
...,...,...,...,...,...
13168509,pswissler,3.0,8256,https://boardgamegeek.com/user/pswissler,NaN
13168510,Sprayoncrayon,3.0,8256,https://boardgamegeek.com/user/Sprayoncrayon,Canada
13168511,PhoenixSong,2.5,8256,https://boardgamegeek.com/user/PhoenixSong,United States
13168512,tsantos,2.0,8256,https://boardgamegeek.com/user/tsantos,United States


In [93]:
#Remove leading/trailing whitespace
ratings_by_country['country'] = ratings_by_country['country'].str.strip()

#Drop rows where country is null
ratings_by_country = ratings_by_country.dropna(subset=['country'])

#Drop rows where country is an empty string
ratings_by_country = ratings_by_country[ratings_by_country['country'] != '']

ratings_by_country

,users,rating,ID,url,country
0,sidehacker,10.0,13,https://boardgamegeek.com/user/sidehacker,United States
1,Varthlokkur,10.0,13,https://boardgamegeek.com/user/Varthlokkur,United States
2,dougthonus,10.0,13,https://boardgamegeek.com/user/dougthonus,United States
3,cypar7,10.0,13,https://boardgamegeek.com/user/cypar7,United States
4,ssmooth,10.0,13,https://boardgamegeek.com/user/ssmooth,United States
...,...,...,...,...,...
13168508,asm71,3.0,8256,https://boardgamegeek.com/user/asm71,United States
13168510,Sprayoncrayon,3.0,8256,https://boardgamegeek.com/user/Sprayoncrayon,Canada
13168511,PhoenixSong,2.5,8256,https://boardgamegeek.com/user/PhoenixSong,United States
13168512,tsantos,2.0,8256,https://boardgamegeek.com/user/tsantos,United States


In [94]:
#Double check: all ratings in ratings_by_countries are of games in games19, other ones are irrelevant to our analysis
ratings_by_country = ratings_by_country[ratings_by_country['ID'].isin(games19.index)]
ratings_by_country

,users,rating,ID,url,country
0,sidehacker,10.0,13,https://boardgamegeek.com/user/sidehacker,United States
1,Varthlokkur,10.0,13,https://boardgamegeek.com/user/Varthlokkur,United States
2,dougthonus,10.0,13,https://boardgamegeek.com/user/dougthonus,United States
3,cypar7,10.0,13,https://boardgamegeek.com/user/cypar7,United States
4,ssmooth,10.0,13,https://boardgamegeek.com/user/ssmooth,United States
...,...,...,...,...,...
13168508,asm71,3.0,8256,https://boardgamegeek.com/user/asm71,United States
13168510,Sprayoncrayon,3.0,8256,https://boardgamegeek.com/user/Sprayoncrayon,Canada
13168511,PhoenixSong,2.5,8256,https://boardgamegeek.com/user/PhoenixSong,United States
13168512,tsantos,2.0,8256,https://boardgamegeek.com/user/tsantos,United States


In [95]:
#How many ratings do we have per game from each country?
users_ratings_per_country = ratings_by_country.groupby(['ID', 'country']).agg('count').reset_index()
users_ratings_per_country

,ID,country,users,rating,url
0,1,APO/FPO,1,1,1
1,1,Argentina,4,4,4
2,1,Australia,95,95,95
3,1,Austria,31,31,31
4,1,Belarus,1,1,1
...,...,...,...,...,...
391185,272409,Greece,2,2,2
391186,272409,Portugal,1,1,1
391187,272409,Spain,2,2,2
391188,272409,United Kingdom,1,1,1


## Data Processing for Visualizations

### Figure 1

In [96]:
games19_top3 = games19.nsmallest(3, 'rank')
games22_top3 = games22.nsmallest(3, 'rank')
games25_top3 = games25.nsmallest(3, 'rank')

In [ ]:
games19_top3.to_csv('../data/games2019_top3.csv')
games22_top3.to_csv('../data/games2022_top3.csv')
games25_top3.to_csv('../data/games2025_top3.csv')

In [ ]:
games19_top3.to_json("../data/games2019_top3.json", orient="records", indent=2)
games22_top3.to_json("../data/games2022_top3.json", orient="records", indent=2)
games25_top3.to_json("../data/games2025_top3.json", orient="records", indent=2)

### Figure 2

For our specific case, we care about the genres of the games (war, sci-fi...), not the mechanics of the game (card game, party game...)

1. Remove mechanics
2. Combine sub-genres under the same genre

In [99]:
def get_most_common_category_patterns(df, category_column):
    if category_column not in df.columns:
        raise ValueError(f"Column '{category_column}' not found in DataFrame.")
    pattern_counts = (
        df[category_column]
        .dropna()
        .apply(lambda x: tuple(sorted(x)))
        .value_counts()
    )
    return pattern_counts


def filter_mechanics_to_new_column(df, category_column, mechanics_to_remove, new_column='filtered_categories'):
    df[new_column] = df[category_column].apply(
        lambda x: [item for item in x if item not in mechanics_to_remove] if isinstance(x, list) else x
    )
    return df


def apply_genre_groups(df, category_column, genre_groups, new_column='grouped_categories'):
    genre_lookup = {
        genre: group for group, genres in genre_groups.items() for genre in genres
    }

    def map_genres(genres):
        if not isinstance(genres, list):
            return genres
        grouped = [genre_lookup.get(g, g) for g in genres]
        seen = set()
        return [g for g in grouped if not (g in seen or seen.add(g))]

    df[new_column] = df[category_column].apply(map_genres)
    return df

In [ ]:
mechanics_to_remove = {
    'Card Game', 'Dice', 'Party Game', 'Bluffing', 'Abstract Strategy', 'Children\'s Game', 'Memory', 'Trivia',
    'Word Game', 'Real-time', 'Negotiation', 'Puzzle', 'Player Elimination', 'Collectible Components', 'Hand Management', 'Print & Play',
    'Game System', 'Expansion for Base-game', 'Novel-based', 'Miniatures', 'Video Game Theme', 'Electronic', 'Comic Book / Strip', 'Maze',
    'Mature / Adult', 'Fan Expansion'
}

genre_groups = {
    'Fantasy': ['Fantasy', 'Mythology', 'Pirates', 'Zombies'],
    'Science Fiction': ['Science Fiction', 'Space Exploration', 'Medical'],
    'History': ['Medieval', 'Renaissance', 'Ancient', 'Civilization',
                                'American West', 'Historical', 'Arabian', 'Napoleonic', 'Post-Napoleonic', 'Prehistoric',
                                'Wargame', 'World War I', 'World War II', 'Modern Warfare', 'Vietnam War',
                                'Civil War', 'American Civil War', 'Territory Building', 'American Revolutionary War',
                                 'American Indian Wars', 'Korean War', 'Pike and Shot', 'Age of Reason'], #History/Civilization
    'Thriller': ['Horror', 'Murder / Mystery', 'Murder/Mystery', 'Spies/Secret Agents', 'Mystery', 'Deduction', 'Spies / Secret Agents'],
    'Action/Adventure': ['Adventure', 'Exploration', 'Nautical', 'Travel', 'Action / Dexterity'],
    'Economic': ['Economic', 'City Building', 'Farming', 'Transportation', 'Industry / Manufacturing', 'Trains', 'Aviation / Flight'],
    'Social': ['Political', 'Mafia', 'Environmental', 'Religious', 'Humor'],
    'Education': ['Educational', 'Math', 'Number'],
    'Arts': ['Arts', 'Movies / TV / Radio theme', 'Music', 'Book'],
    'Sports': ['Racing', 'Sports']
}

In [200]:
games19_categ = filter_mechanics_to_new_column(games19, 'boardgamecategory', mechanics_to_remove, new_column='filtered_categories')
get_most_common_category_patterns(games19_categ, 'filtered_categories')

filtered_categories
()                                                                                   3039
(Wargame, World War II)                                                               775
(Fantasy,)                                                                            473
(Action / Dexterity,)                                                                 441
(Animals,)                                                                            387
                                                                                     ... 
(Nautical, Racing, Trains, Transportation)                                              1
(Educational, Modern Warfare, Wargame)                                                  1
(City Building, Civilization, Economic, Industry / Manufacturing, Transportation)       1
(Deduction, Medieval, Spies/Secret Agents)                                              1
(Adventure, Deduction)                                                          

In [201]:
apply_genre_groups(games19_categ, 'filtered_categories', genre_groups)
games19_grouped = get_most_common_category_patterns(games19_categ, 'grouped_categories')
games19_grouped

grouped_categories
()                                                                                3039
(Historical/Civilization,)                                                        2670
(Economic,)                                                                        877
(Action/Adventure,)                                                                675
(Fantasy,)                                                                         657
                                                                                  ... 
(Action/Adventure, Animals, Fighting, Sports)                                        1
(Education, Historical/Civilization, Science Fiction, Social)                        1
(Action/Adventure, Economic, Historical/Civilization, Science Fiction, Social)       1
(Action/Adventure, Economic, Fantasy, Fighting, Thriller)                            1
(Arts, Education, Sports)                                                            1
Name: count, Length: 403

In [202]:
games22_categ = filter_mechanics_to_new_column(games22, 'boardgamecategory', mechanics_to_remove, new_column='filtered_categories')
get_most_common_category_patterns(games22_categ, 'filtered_categories')

filtered_categories
()                                                                                          3890
(Wargame, World War II)                                                                      893
(Fantasy,)                                                                                   625
(Action / Dexterity,)                                                                        535
(Animals,)                                                                                   525
                                                                                            ... 
(Farming, Industry / Manufacturing, Medieval, Territory Building)                              1
(Adventure, Economic, Political, Science Fiction)                                              1
(City Building, Civilization, Exploration, Industry / Manufacturing, Territory Building)       1
(Adventure, Fantasy, Nautical, Pirates)                                                        1
(American 

In [203]:
apply_genre_groups(games22_categ, 'filtered_categories', genre_groups)
games22_grouped = get_most_common_category_patterns(games22_categ, 'grouped_categories')
games22_grouped

grouped_categories
()                                                                                       3890
(Historical/Civilization,)                                                               3123
(Economic,)                                                                              1058
(Fantasy,)                                                                                850
(Action/Adventure,)                                                                       836
                                                                                         ... 
(Action/Adventure, Economic, Fantasy, Fighting, Thriller)                                   1
(Action/Adventure, Arts, Fantasy, Historical/Civilization, Science Fiction, Thriller)       1
(Action/Adventure, Arts, Economic, Historical/Civilization, Science Fiction, Social)        1
(Arts, Economic, Fantasy)                                                                   1
(Animals, Arts, Education)               

In [204]:
games25_categ = filter_mechanics_to_new_column(games25, 'boardgamecategory', mechanics_to_remove, new_column='filtered_categories')
get_most_common_category_patterns(games25_categ, 'filtered_categories')

filtered_categories
()                                                                                                        5200
(Wargame, World War II)                                                                                   1029
(Fantasy,)                                                                                                 812
(Animals,)                                                                                                 753
(Action / Dexterity,)                                                                                      664
                                                                                                          ... 
(Deduction, Educational, Movies / TV / Radio theme, Prehistoric)                                             1
(American West, Mythology)                                                                                   1
(Ancient, Economic, Farming)                                                                

In [205]:
apply_genre_groups(games25_categ, 'filtered_categories', genre_groups)
games25_grouped = get_most_common_category_patterns(games25_categ, 'grouped_categories')
games25_grouped

grouped_categories
()                                                                                      5200
(Historical/Civilization,)                                                              3697
(Economic,)                                                                             1299
(Fantasy,)                                                                              1114
(Action/Adventure,)                                                                     1090
                                                                                        ... 
(Arts, Education, Science Fiction)                                                         1
(Action/Adventure, Economic, Fantasy, Fighting, Thriller)                                  1
(Action/Adventure, Arts, Economic, Historical/Civilization, Science Fiction, Social)       1
(Action/Adventure, Fighting, Historical/Civilization, Sports)                              1
(Action/Adventure, Animals, Arts, Education, Social

In [206]:
multi_category = games19_grouped[games19_grouped.index.to_series().apply(lambda x: len(x) > 1)]
print(f"Number of multi-category entries: {len(multi_category)}")
multi_category

Number of multi-category entries: 390


grouped_categories
(Economic, Historical/Civilization)                                               424
(Action/Adventure, Fantasy)                                                       286
(Action/Adventure, Historical/Civilization)                                       285
(Fantasy, Historical/Civilization)                                                256
(Historical/Civilization, Science Fiction)                                        212
                                                                                 ... 
(Action/Adventure, Animals, Fighting, Sports)                                       1
(Education, Historical/Civilization, Science Fiction, Social)                       1
(Action/Adventure, Economic, Historical/Civilization, Science Fiction, Social)      1
(Action/Adventure, Economic, Fantasy, Fighting, Thriller)                           1
(Arts, Education, Sports)                                                           1
Name: count, Length: 390, dtype: in

In [197]:
def get_normalized_category_contributions(grouped_categories: pd.Series) -> pd.DataFrame:

    #Remove entries with empty category tuples
    non_empty = grouped_categories[grouped_categories.index.to_series().apply(lambda x: len(x) > 0)]

    #Distribute count evenly to each category in the tuple
    category_contributions = Counter()
    for cats, count in non_empty.items():
        per_cat_weight = count / len(cats)
        for cat in cats:
            category_contributions[cat] += per_cat_weight

    cat_df = pd.DataFrame.from_dict(category_contributions, orient='index', columns=['contribution'])
    cat_df['proportion'] = cat_df['contribution'] / cat_df['contribution'].sum()
    cat_df = cat_df.sort_values('proportion', ascending=False)

    return cat_df

In [ ]:
grouped = {
    '2019': games19_grouped,
    '2022': games22_grouped,
    '2025': games25_grouped,
}

for year, grouped_data in grouped.items():
    df = get_normalized_category_contributions(grouped_data)
    df.to_csv(f'../data/categories_{year}.csv')
    df.to_json(f'../data/categories_{year}.json', orient='index', indent=2)

### Figure 3

In [62]:
def compute_board_game_properties(df, year_label):
    return {
        "Year": year_label,
        "Average Weight": df['averageweight'].mean(),
        "Average Minimum Players": df['minplayers'].mean(),
        "Solo Playable (%)": (df['minplayers'] == 1).mean() * 100,
        "Average Minimum Age": df['minage'].mean()
    }

In [ ]:
props_2019 = compute_board_game_properties(games19, "2019")
props_2022 = compute_board_game_properties(games22, "2022")
props_2025 = compute_board_game_properties(games25, "2025")

properties_df = pd.DataFrame([props_2019, props_2022, props_2025])
properties_df.to_csv("../data/board_game_properties_over_time.csv", index=False)